In [1]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn import svm
import numpy as np
from sklearn.datasets.samples_generator import make_blobs
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

C:\Users\Álvaro\anaconda3.1\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.datasets.samples_generator module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
df = pd.read_csv(r"C:\Users\Álvaro\Final-project\Data\data.csv")

In [3]:
df

,eventTime_ay,Date,email,firstName,lastName,Sub_Categoria,campaign,medium,userId,source,...,time_diff_ay_ac,time_diff_ac_vg,time_diff_vg_ff,time_diff_ff_fd,time_diff_fd_pm,city,cityId,region,regionId,sessions
0,2020-05-09 10:19:12,2020-05-09,eduardoperez@seincomobiliario.com,EDUARDO,PEREZ,disenador-de-muebles,NaN,NaN,083e8b2f-af6a-46b4-a9b9-4f79c1a32cf1,NaN,...,NaN,NaN,0 days 00:00:03,NaN,NaN,Barcelona,1005424,Catalonia,20278,2.0
1,2020-04-12 14:50:10,2020-04-12,alexvonder@hotmail.com,Alexandra,Perez,chofer,01_España_Genéricas,ppc,0feedfe0-5452-434c-8a70-845fa3fd638a,adwords,...,NaN,NaN,0 days 00:00:02,NaN,NaN,(not set),(not set),Balearic Islands,21387,6.0
2,2020-11-08 18:09:23,2020-11-08,Evo@gmail.com,Juanjo,Akze,fotoperiodista,10430315940,cpc,18bb05eb-d592-4459-a682-f622afb7208c,adwords,...,NaN,NaN,NaN,NaN,NaN,Mollet del Valles,1005431,Catalonia,20278,2.0
3,2020-08-25 15:18:38,2020-08-25,cesardelacalle@grupoccs.es,cesar fernando,de la calle verde,conserje,10430285424,cpc,328d85a1-28b3-493d-ad17-ab58e4567357,adwords,...,NaN,NaN,NaN,NaN,NaN,Villanueva de la Canada,9049240,Madrid,9047045,1.0
4,2020-11-16 11:02:42,2020-11-16,brsbrs02@gmail.com,Bruno,RS,proyectista,10430285424,cpc,39d91a35-e406-462d-a7a5-7b2291ba8340,adwords,...,NaN,NaN,NaN,NaN,NaN,A Coruna,1005479,Galicia,20280,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8057,2020-07-05 11:44:34,2020-07-05,kk@hotmail.com,Santi,Mirlo,entrenador-o-monitor-deportivo,NaN,NaN,7a6ca821-1e99-4156-bb35-9d904828b26b,NaN,...,NaN,NaN,0 days 00:00:03,NaN,NaN,Castellon de la Plana,1005539,Valencian Community,21388,1.0
8058,2020-08-29 08:45:46,2020-08-29,sarah@slyne.com,Sarah,Slyne,entrenador-o-monitor-deportivo,NaN,NaN,ab772667-56c1-4e2a-b938-ae6e4e3980e3,NaN,...,NaN,NaN,NaN,NaN,NaN,Seville,1005421,Andalusia,20269,2.0
8059,2020-06-04 09:03:30,2020-06-04,saraculebras@gmail.com,Sara,Culebras Yagüe,entrenador-o-monitor-deportivo,NaN,NaN,dc059f05-6e2a-4cb6-9436-826b38d87ce9,NaN,...,NaN,NaN,0 days 00:00:03,NaN,NaN,Torrejon de Ardoz,1005496,Madrid,9047045,1.0
8060,2020-09-30 16:03:25,2020-09-30,blancopris@gmail.com,Priscila,Abbona,entrenador-o-monitor-deportivo,RC_pillarpage_nurturing,email,e2af6a50-4e9e-42c0-a5fc-784141160d31,hs_automation,...,NaN,NaN,0 days 00:00:04,NaN,NaN,Pamplona,1005503,Navarre,9047051,1.0


We take out regions that are not from Spain

In [4]:
data = df.sort_values(by = 'Date', ascending = True)

In [5]:
df["region"].replace(' ', '_', regex=True, inplace=True)

In [6]:
df = df[~df['region'].isin(["Ile-de-France","(not_set)","Occitanie","England","Bogota","Bavaria","North_Rhine-Westphalia","Porto_District","Santiago_Metropolitan_Region","Morelos","Melilla","Scotland","Andorra_la_Vella","Zulia","Prague","Lombardy","Hessen","New_York","Capital_District","North_Holland","Occitania","Azuay","Cartago_Province","Utrecht","New_Hampshire","Pays_de_la_Loire","Georgia","Santa_Fe_Province","Buenos_Aires","Rio_Grande","Vienna"])]

In [7]:
df2 = df[[ 'categorySlug', 'region', 'RC', 'Comercio', 'sale']]

Dropna columns

In [8]:
df3=df2.dropna()

# Feature engineering:

In [9]:
def get_dummies(df, varlist):
    for var in varlist:
        df_slice = pd.get_dummies(df[var])
        df = pd.concat([df.drop(var, axis =1), df_slice], axis =1)
    return df

In [10]:
categorical_list = ["categorySlug", "region"]

In [11]:
categorical_list = ["categorySlug","region"]
X = df3.drop(columns='sale')
X = get_dummies(X, categorical_list)
y = df3['sale']


In [12]:
df3['sale'].value_counts()

0    6456
1     833
Name: sale, dtype: int64

In [19]:
from imblearn.combine import SMOTEENN 
from imblearn.combine import SMOTETomek

oversamp_undersamp = SMOTETomek(sampling_strategy=0.6) #default sampling strategy
X_train_b, y_train_b = oversamp_undersamp.fit_sample(X_train, y_train)
X_train_b.shape, y_train_b.shape
y_train_b.value_counts()

0    4502
1    2701
Name: sale, dtype: int64

In [20]:
model = RandomForestClassifier()


In [21]:
model.fit(X_train_b, np.ravel(y_train_b)).score(X_test, np.ravel(y_test))

0.81572930955647

In [22]:
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [23]:
rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, 
                               param_distributions = random_grid, 
                               n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

In [24]:
rf_random.fit(X_train_b, y_train_b)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   55.2s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  4.4min


OSError: [Errno 22] Invalid argument

In [ ]:
print('Random Forest classifier best paramas')
print(rf_random.best_score_)
print(rf_random.best_params_)

In [ ]:
rf_optimal = RandomForestClassifier(n_estimators = 1600, min_samples_split = 10,
                                    min_samples_leaf = 1, max_features = 'sqrt', max_depth = None,
                                    bootstrap = True)

In [ ]:
rf_optimal.fit(X_train_b, y_train_b)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

predictions_train = rf_optimal.predict(X_train_b)
predictions_test = rf_optimal.predict(X_test)
print(classification_report(y_test, predictions_test))
print("accuracy score on test:", accuracy_score(y_test, predictions_test))
print("accuracy on train:", accuracy_score(y_train_b, predictions_train))

In [ ]:
df['Month'] = pd.DatetimeIndex(df['Date']).month

We include month as a variable

In [ ]:
df4 = df[[ "Month",'categorySlug', 'region', 'RC', 'Comercio', 'sale']]
df4d

In [ ]:
df4=df4.dropna()

In [ ]:
categorical_list = ["categorySlug","region","Month"]
X = df4.drop(columns='sale')
X = get_dummies(X, categorical_list)
y = df4['sale']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3)

In [ ]:
df4['sale'].value_counts()

In [ ]:
from imblearn.combine import SMOTEENN 
from imblearn.combine import SMOTETomek

oversamp_undersamp = SMOTETomek(sampling_strategy=0.6) #default sampling strategy
X_train_b, y_train_b = oversamp_undersamp.fit_sample(X_train, y_train)
X_train_b.shape, y_train_b.shape
y_train_b.value_counts()

In [ ]:
model = RandomForestClassifier()

In [ ]:
model.fit(X_train_b, np.ravel(y_train_b)).score(X_test, np.ravel(y_test))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, 
                               param_distributions = random_grid, 
                               n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

In [ ]:
rf_random.fit(X_train_b, y_train_b)

In [ ]:
print('Random Forest classifier best paramas')
print(rf_random.best_score_)
print(rf_random.best_params_)

In [ ]:
rf_optimal = RandomForestClassifier(n_estimators = 1800, min_samples_split = 10,
                                    min_samples_leaf = 1, max_features = 'sqrt', max_depth = 30,
                                    bootstrap = False)

In [ ]:
rf_optimal.fit(X_train_b, y_train_b)

In [ ]:
#Month added
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

predictions_train = rf_optimal.predict(X_train_b)
predictions_test = rf_optimal.predict(X_test)
print(classification_report(y_test, predictions_test))
print("accuracy score on test:", accuracy_score(y_test, predictions_test))
print("accuracy on train:", accuracy_score(y_train_b, predictions_train))

We include source as a new variable

In [ ]:
df5 = df[['categorySlug', 'region', 'RC', 'Comercio', 'sale',"source","Month"]]

df5=df5.dropna()

In [ ]:
categorical_list = ["categorySlug","region","source","Month"]
X = df5.drop(columns='sale')
X = get_dummies(X, categorical_list)
y = df5['sale']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3)

In [ ]:
df5=df5.dropna(inplace=True)

In [ ]:
oversamp_undersamp = SMOTETomek(sampling_strategy=0.6) #default sampling strategy
X_train_b, y_train_b = oversamp_undersamp.fit_sample(X_train, y_train)
X_train_b.shape, y_train_b.shape
y_train_b.value_counts()

In [ ]:
model = RandomForestClassifier()

In [ ]:
model.fit(X_train_b, np.ravel(y_train_b)).score(X_test, np.ravel(y_test))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, 
                               param_distributions = random_grid, 
                               n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

In [ ]:
rf_random.fit(X_train_b, y_train_b)

In [ ]:
print('Random Forest classifier best paramas')
print(rf_random.best_score_)
print(rf_random.best_params_)

In [ ]:
rf_optimal = RandomForestClassifier(n_estimators = 1800, min_samples_split = 10,
                                    min_samples_leaf = 1, max_features = 'sqrt', max_depth = 30,
                                    bootstrap = False)

In [ ]:
rf_optimal.fit(X_train_b, y_train_b)

In [ ]:
#Month added
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

predictions_train = rf_optimal.predict(X_train_b)
predictions_test = rf_optimal.predict(X_test)
print(classification_report(y_test, predictions_test))
print("accuracy score on test:", accuracy_score(y_test, predictions_test))
print("accuracy on train:", accuracy_score(y_train_b, predictions_train))

We include sessions as a new variable

In [ ]:
df6 = df[['categorySlug', 'region', 'RC', 'Comercio', 'sale',"source","Month","sessions"]]

df6=df6.dropna()

In [ ]:
categorical_list = ["categorySlug","region","source","Month"]
X = df6.drop(columns='sale')
X = get_dummies(X, categorical_list)
y = df6['sale']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3)

In [ ]:
oversamp_undersamp = SMOTETomek(sampling_strategy=0.6) #default sampling strategy
X_train_b, y_train_b = oversamp_undersamp.fit_sample(X_train, y_train)
X_train_b.shape, y_train_b.shape
y_train_b.value_counts()

In [ ]:
model = RandomForestClassifier()

In [ ]:
model.fit(X_train_b, np.ravel(y_train_b)).score(X_test, np.ravel(y_test))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, 
                               param_distributions = random_grid, 
                               n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

In [ ]:
rf_random.fit(X_train_b, y_train_b)

In [ ]:
print('Random Forest classifier best paramas')
print(rf_random.best_score_)
print(rf_random.best_params_)

In [ ]:
rf_optimal = RandomForestClassifier(n_estimators = 1800, min_samples_split = 10,
                                    min_samples_leaf = 1, max_features = 'sqrt', max_depth = 30,
                                    bootstrap = False)

In [ ]:
rf_optimal.fit(X_train_b, y_train_b)

In [ ]:
#Month added
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

predictions_train = rf_optimal.predict(X_train_b)
predictions_test = rf_optimal.predict(X_test)
print(classification_report(y_test, predictions_test))
print("accuracy score on test:", accuracy_score(y_test, predictions_test))
print("accuracy on train:", accuracy_score(y_train_b, predictions_train))

We include weekday a as variable


In [ ]:
df['WEEKDAY'] = pd.to_datetime(df['Date']).dt.dayofweek

In [ ]:
df7 = df[['categorySlug', 'region', 'RC', 'Comercio', 'sale',"source","Month","sessions","WEEKDAY"]]

df7=df7.dropna()

In [ ]:
categorical_list = ["categorySlug","region","source","Month"]
X = df7.drop(columns='sale')
X = get_dummies(X, categorical_list)
y = df7['sale']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3)

In [ ]:
oversamp_undersamp = SMOTETomek(sampling_strategy=0.6) #default sampling strategy
X_train_b, y_train_b = oversamp_undersamp.fit_sample(X_train, y_train)
X_train_b.shape, y_train_b.shape
y_train_b.value_counts()

In [ ]:
model = RandomForestClassifier()

In [ ]:
model.fit(X_train_b, np.ravel(y_train_b)).score(X_test, np.ravel(y_test))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, 
                               param_distributions = random_grid, 
                               n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)

In [ ]:
rf_random.fit(X_train_b, y_train_b)

In [ ]:
print('Random Forest classifier best paramas')
print(rf_random.best_score_)
print(rf_random.best_params_)

In [ ]:
rf_optimal = RandomForestClassifier(n_estimators = 1800, min_samples_split = 10,
                                    min_samples_leaf = 1, max_features = 'sqrt', max_depth = 30,
                                    bootstrap = False)

In [ ]:
rf_optimal.fit(X_train_b, y_train_b)

In [ ]:
#Month added
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

predictions_train = rf_optimal.predict(X_train_b)
predictions_test = rf_optimal.predict(X_test)
print(classification_report(y_test, predictions_test))
print("accuracy score on test:", accuracy_score(y_test, predictions_test))
print("accuracy on train:", accuracy_score(y_train_b, predictions_train))

In [ ]:
from sklearn.metrics import r2_score
from rfpimp import permutation_importances

def r2(rf, X_train, y_train):
    return r2_score(y_train, rf.predict(X_train))

perm_imp_rfpimp = permutation_importances(rf, X_train, y_train, r2)

Comercio leads prediction

In [ ]:
df8 = df[['categorySlug', 'region','Comercio', 'sale',"source","sessions","quote"]]

df8=df8.dropna()

In [ ]:
categorical_list = ["categorySlug","region","source"]
X = df8.drop(columns='sale')
X = get_dummies(X, categorical_list)
y = df8['sale']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3)

In [ ]:
oversamp_undersamp = SMOTETomek(sampling_strategy=0.6) #default sampling strategy
X_train_b, y_train_b = oversamp_undersamp.fit_sample(X_train, y_train)
X_train_b.shape, y_train_b.shape
y_train_b.value_counts()

In [ ]:
model = RandomForestClassifier()

In [ ]:
model.fit(X_train_b, np.ravel(y_train_b)).score(X_test, np.ravel(y_test))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, 
                               param_distributions = random_grid, 
                               n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)


In [ ]:
rf_random.fit(X_train_b, y_train_b)

In [ ]:
print('Random Forest classifier best paramas')
print(rf_random.best_score_)
print(rf_random.best_params_)

In [ ]:
rf_optimal = RandomForestClassifier(n_estimators = 1800, min_samples_split = 10,
                                    min_samples_leaf = 1, max_features = 'sqrt', max_depth = 30,
                                    bootstrap = False)

In [ ]:
rf_optimal.fit(X_train_b, y_train_b)

In [ ]:
#Month added
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

predictions_train = rf_optimal.predict(X_train_b)
predictions_test = rf_optimal.predict(X_test)
print(classification_report(y_test, predictions_test))
print("accuracy score on test:", accuracy_score(y_test, predictions_test))
print("accuracy on train:", accuracy_score(y_train_b, predictions_train))

In [ ]:
from sklearn.metrics import r2_score
from rfpimp import permutation_importances

def r2(rf, X_train, y_train):
    return r2_score(y_train, rf.predict(X_train))

perm_imp_rfpimp = permutation_importances(rf, X_train, y_train, r2)

In [ ]:
perm_imp_rfpimp

In [ ]:
df9 = df[[ 'categorySlug', 'region', 'RC', 'Comercio', 'sale',"source","session_count_email","session_count_userId","quote","sessions"]]
df9=df9.dropna()

In [ ]:
categorical_list = ["categorySlug","region","source","Month"]
X = df9.drop(columns='sale')
X = get_dummies(X, categorical_list)
y = df9['sale']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3)

In [ ]:
oversamp_undersamp = SMOTETomek(sampling_strategy=0.6) #default sampling strategy
X_train_b, y_train_b = oversamp_undersamp.fit_sample(X_train, y_train)
X_train_b.shape, y_train_b.shape
y_train_b.value_counts()

In [ ]:
model = RandomForestClassifier()

In [ ]:
model.fit(X_train_b, np.ravel(y_train_b)).score(X_test, np.ravel(y_test))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, 
                               param_distributions = random_grid, 
                               n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)


In [ ]:
rf_random.fit(X_train_b, y_train_b)

In [ ]:
print('Random Forest classifier best paramas')
print(rf_random.best_score_)
print(rf_random.best_params_)

In [ ]:
rf_optimal = RandomForestClassifier(n_estimators = 600, min_samples_split = 10,
                                    min_samples_leaf = 1, max_features = 'sqrt', max_depth = 40,
                                    bootstrap = False)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

predictions_train = rf_optimal.predict(X_train_b)
predictions_test = rf_optimal.predict(X_test)
print(classification_report(y_test, predictions_test))
print("accuracy score on test:", accuracy_score(y_test, predictions_test))
print("accuracy on train:", accuracy_score(y_train_b, predictions_train))

In [ ]:
# Limit depth of tree to 3 levels
rf_small = RandomForestRegressor(n_estimators=10, max_depth = 3)
rf_small.fit(train_features, train_labels)
# Extract the small tree
tree_small = rf_small.estimators_[5]
# Save the tree as a png image
export_graphviz(tree_small, out_file = 'small_tree.dot', feature_names = feature_list, rounded = True, precision = 1)
(graph, ) = pydot.graph_from_dot_file('small_tree.dot')
graph.write_png('small_tree.png');

In [ ]:
# Get numerical feature importances
importances = list(rf.feature_importances_)
# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];


In [ ]:
# Import matplotlib for plotting and use magic command for Jupyter Notebooks
import matplotlib.pyplot as plt
%matplotlib inline
# Set the style
plt.style.use('fivethirtyeight')
# list of x locations for plotting
x_values = list(range(len(importances)))
# Make a bar chart
plt.bar(x_values, importances, orientation = 'vertical')
# Tick labels for x axis
plt.xticks(x_values, feature_list, rotation='vertical')
# Axis labels and title
plt.ylabel('Importance'); plt.xlabel('Variable'); plt.title('Variable Importances');